# RestFul API - FastAPI


<img style="display: block; margin: auto;" src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" title="git" width="580" height="50">

[FastAPI](https://fastapi.tiangolo.com/) es un web framework moderno y rápido (de alto rendimiento) para construir APIs con `Python 3.7+` basado en las anotaciones de tipos estándar de `Python`.

Sus características principales son:

- **Rapidez:** Alto rendimiento, a la par con `NodeJS` y `Go` (gracias a Starlette y Pydantic). Uno de los frameworks de Python más rápidos.
- **Rápido de programar:** Incrementa la velocidad de desarrollo entre 200% y 300%.
- **Menos errores:** Reduce los errores humanos (de programador) aproximadamente un 40%.
- **Intuitivo:** Gran soporte en los editores con auto completado en todas partes. Gasta menos tiempo debugging.
- **Fácil:** Está diseñado para ser fácil de usar y aprender. Gastando menos tiempo leyendo documentación.
- **Corto:** Minimiza la duplicación de código. Múltiples funcionalidades con cada declaración de parámetros. Menos errores.
- **Robusto:** Crea código listo para producción con documentación automática interactiva.
- **Basado en estándares:** Basado y totalmente compatible con los estándares abiertos para APIs: OpenAPI (conocido previamente como Swagger) y JSON Schema.

**FastAPI** está ganando popularidad entre los Frameworks de `Python`. Su sintaxis también es similar a la de `Flask`, por lo que es fácil cambiar a ella si ha usado `Flask` antes.

[https://christophergs.com/python/2021/06/16/python-flask-fastapi/](https://christophergs.com/python/2021/06/16/python-flask-fastapi/)

![](https://christophergs.com/assets/images/fastapi_flask_post/benchmarks.jpeg)

## Conceptos clave: Path, Query y Body Parameters

### 1. Path Parameters:

- Se utilizan para capturar valores directamente desde la ruta del endpoint.
- Ejemplo:
```python
@app.get("/items/{item_id}")
async def read_item(item_id: int):
   return {"item_id": item_id}
```
- **Cuándo usarlo:** Cuando el valor es parte de la estructura de la URL, como un identificador de recurso.

### 2. Query Parameters:

- Se envían en la URL después del signo `?` y se usan para filtrar, buscar o modificar la respuesta.
- Ejemplo:
```python
@app.get("/items/")
async def read_items(skip: int = 0, limit: int = 10):
   return {"skip": skip, "limit": limit}
```
- **Cuándo usarlo:** Para opciones de filtro, paginación, búsqueda o parámetros opcionales.

### 3. Request Body:

- Se usan para enviar datos más complejos (como JSON) en el cuerpo de la solicitud.
- Con FastAPI, se definen usando clases Pydantic.
- Ejemplo:
```python
from pydantic import BaseModel

class Item(BaseModel):
   name: str
   description: str = None
   price: float
   tax: float = None

@app.post("/items/")
async def create_item(item: Item):
   return item
```
- **Cuándo usarlo:** Cuando se necesitan enviar datos estructurados y complejos que no caben en una URL o consulta.

## Uso de Pydantic para Definir Tipos como Clases
### 1. ¿Qué es Pydantic?

Pydantic permite la validación de datos basados en tipos, convirtiendo automáticamente tipos de datos, validando estructuras complejas, y generando documentación de forma automática.

### 2. Ejemplo práctico:

 ```python
 from pydantic import BaseModel

 class User(BaseModel):
     username: str
     email: str
     full_name: str = None
 ```
   
### 3. Ventajas de usar Pydantic:
- Validación automática y robusta.
- Conversión de tipos.
- Generación automática de documentación.

## Nombramiento de Endpoints y Relación con Recursos en REST APIs

### 1. Endpoints en FastAPI:
- Deben ser nombrados en función de los recursos que gestionan.
- Ejemplos de buenas prácticas:
    - `/items/` para gestionar items.
    - `/users/` para gestionar usuarios.
    - `/items/{item_id}/` para acceder a un recurso específico.



## Autenticación con API Key y Uso de Variables de Entorno
### 1. Implementación de API Key en FastAPI:
```python
from fastapi import FastAPI, Depends, HTTPException, Security
from fastapi.security.api_key import APIKeyQuery
import os

app = FastAPI()

API_KEY = os.getenv("API_KEY")
API_KEY_NAME = "access_token"
api_key_query = APIKeyQuery(name=API_KEY_NAME)

async def get_api_key(api_key_query: str = Security(api_key_query)):
 if api_key_query == API_KEY:
     return api_key_query
 else:
     raise HTTPException(
         status_code=403, detail="Could not validate credentials"
     )

@app.get("/secure-data/")
async def secure_data(api_key: str = Depends(get_api_key)):
 return {"message": "Secure data access granted."}
```

- **Explicación del código:**
    - **APIKeyQuery:** Se utiliza para definir un parámetro de consulta (`query parameter`) que se espera en la URL para la autenticación.
    - **Dependencias:** FastAPI usa dependencias (`Depends`) para verificar la API Key antes de procesar la solicitud.
    - **Variables de entorno:** La API Key se carga desde una variable de entorno (`os.getenv("API_KEY")`).

- **Cómo cargar variables de entorno:**
    - Crear un archivo `.env`:
    ```
    API_KEY=your_secure_api_key
    ```
    - Usar `python-dotenv` para cargar las variables de entorno en tu aplicación:
    ```python
    from dotenv import load_dotenv
    load_dotenv()
    ```


## Manos a la obra!
1. Crear repositorio llamado `fastapi-hello-world-o-2024`
2. Clonar el repositorio
3. Crear un ambiente virtual
4. Instalar las librerías necesarias
```bash
pip install "fastapi[standard]" python-dotenv
```

In [ ]:
from dotenv import load_dotenv
from fastapi import FastAPI, Depends, HTTPException, Security
from pydantic import BaseModel
from fastapi.security.api_key import APIKeyQuery
import os

_ = load_dotenv()

app = FastAPI()

API_KEY = os.getenv("API_KEY")
API_KEY_NAME = "access_token"
api_key_query = APIKeyQuery(name=API_KEY_NAME)

async def get_api_key(api_key: str = Security(api_key_query)):
   if api_key == API_KEY:
       return api_key
   else:
       raise HTTPException(
           status_code=403, detail="Could not validate credentials"
       )

class Item(BaseModel):
   name: str
   description: str = None
   price: float
   tax: float = None

@app.get("/items/{item_id}")
async def read_item(item_id: int, q: str = None, api_key: str = Depends(get_api_key)):
   return {"item_id": item_id, "q": q}

@app.post("/items/")
async def create_item(item: Item, api_key: str = Depends(get_api_key)):
   return {"name": item.name, "price": item.price}

## Conectar FastAPI a una base de datos

1. Instalar sqlite3:
   - Windows: https://youtu.be/2CAspm7YwTU?t=74
   - Mac: Instalar brew (Viene por default con brew)
   - Ubuntu: `sudo apt install sqlite3`
   - Para validar, desde terminal correr `sqlite3 --version`

2. Instalar sqlalchemy: `pip install sqlalchemy`
3. Crear los siguientes scripts:

### `books.py`

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, Field
import models
from database import engine, SessionLocal
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)


def get_db():
    try:
        db = SessionLocal()
        yield db
    finally:
        db.close()


class Book(BaseModel):
    title: str = Field(min_length=1)
    author: str = Field(min_length=1, max_length=100)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=-1, lt=101)


@app.get("/")
def read_api(db: Session = Depends(get_db)):
    return db.query(models.Books).all()


@app.post("/")
def create_book(book: Book, db: Session = Depends(get_db)):

    book_model = models.Books()
    book_model.title = book.title
    book_model.author = book.author
    book_model.description = book.description
    book_model.rating = book.rating

    db.add(book_model)
    db.commit()

    return book


@app.put("/{book_id}")
def update_book(book_id: int, book: Book, db: Session = Depends(get_db)):

    book_model = db.query(models.Books).filter(models.Books.id == book_id).first()

    if book_model is None:
        raise HTTPException(
            status_code=404,
            detail=f"ID {book_id} : Does not exist"
        )

    book_model.title = book.title
    book_model.author = book.author
    book_model.description = book.description
    book_model.rating = book.rating

    db.add(book_model)
    db.commit()

    return book


@app.delete("/{book_id}")
def delete_book(book_id: int, db: Session = Depends(get_db)):

    book_model = db.query(models.Books).filter(models.Books.id == book_id).first()

    if book_model is None:
        raise HTTPException(
            status_code=404,
            detail=f"ID {book_id} : Does not exist"
        )

    db.query(models.Books).filter(models.Books.id == book_id).delete()

    db.commit()

### `database.py`

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./books.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

### `models.py`

In [ ]:
from sqlalchemy import Column, Integer, String
from database import Base


class Books(Base):
    __tablename__ = "books"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    author = Column(String)
    description = Column(String)
    rating = Column(Integer)

## Tarea 3

- Crear un nuevo repositorio que se llame `Tarea3_PCD`
- Clonar el repositorio en tu local
- Crear un ambiente virtual
- Crear una nueva API, la cuál contenga cuatro endpoints con las siguientes consideraciones:

    1. Un endpoint para crear un usuario con los siguientes atributos y guardarlos en una tabla en una database:
        ```
        {"user_name": "name",
        "user_id": id,
        "user_email": "email",
        "age" (optiona): age,
        "recommendations": list[str],
        "ZIP" (optional): ZIP
        }
        ```
    
    2. Si se envía datos con un email ya repetido, se debe regresar un mensaje de error que mencione este hecho.
    3. Un endpoint para actualizar la información de un usuario específico buscándolo por id. Si el id no existe, debe regresar un mensaje de error que mencione este hecho.
    4. Un endpoint para obtener la información de un usuario específico buscándolo por id. Si el id no existe, debe regresar un mensaje de error que mencione este hecho.
    5. Un endpoint para eliminar la información de un usuario específico buscándolo por id. Si el id no existe, debe regresar un mensaje de error que mencione este hecho.
    6. Crear dicho endpoint en un archivo llamado `main.py`.

- El repositorio debe tener un archivo `README.MD`
- El proyecto debe contar con un archivo `requirements.txt`

Subir el link del repositorio al espacio programado en Canvas


### FECHA DE ENTREGA: MARTES 3 DE SEPTIEMBRE DE 2024 A LAS 19:55

## QUIZ 2
### FECHA DEL QUIZ: MARTES 3 DE SEPTIEMBRE EN EL HORARIO DE CLASE.
### EL QUIZ ABARCARA LOS TEMAS DE AMBIENTES VIRTUALES, API Y FASTAPI.